<a href="https://colab.research.google.com/github/NINGTANG1124/UPF-HFI/blob/main/notebooks/intake24_nova_matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# connect googledrive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 0. Data Processing

## 0.1 Intake24 Data

In [2]:
# Read intake data (including Descriptionen and FoodGroupen)
import pandas as pd
import re

file_path = "/content/drive/MyDrive/UPF-HFI/Bradford_original data/1. Dietmasterfile_foodlevel_clean.xls"
intake_df = pd.read_excel(file_path)

# Define text cleaning function
def clean_text(col):
    return col.astype(str).str.lower().str.strip().str.replace(r"\s+", " ", regex=True)

# Apply to key fields
intake_df["Foodgroupen_clean"] = clean_text(intake_df["Foodgroupen"])
intake_df["Descriptionen_clean"] = clean_text(intake_df["Descriptionen"])


## 0.2 NOVA Data

In [3]:
# Nova file data cleaning
ndns_df = pd.read_csv("/content/drive/MyDrive/UPF-HFI/nova/NDNS_NOVA_DATABASE.new2023.csv", encoding="ISO-8859-1")
ndns_df.columns = ndns_df.columns.str.strip()
ndns_df = ndns_df[["FoodName", "NOVA"]].dropna()
ndns_df["FoodName_clean"] = ndns_df["FoodName"].str.lower().str.replace(r"[^\w\s]", " ", regex=True).str.replace(r"\s+", " ", regex=True).str.strip()
ndns_df = ndns_df.drop_duplicates(subset=["FoodName_clean"])

# Giulia file data cleaning
giulia_df = pd.read_excel("/content/drive/MyDrive/UPF-HFI/nova/Training Data Original Given by NOVA Researchers - Corrections by Giulia Babak FNDDS 2009-10.xls")
giulia_df.columns = giulia_df.columns.str.strip()

giulia_df = giulia_df[["Main_food_description", "SR_nova_group"]].dropna()
giulia_df = giulia_df.rename(columns={"Main_food_description": "FoodName", "SR_nova_group": "NOVA"})

giulia_df["FoodName_clean"] = giulia_df["FoodName"].str.lower().str.replace(r"[^\w\s]", " ", regex=True).str.replace(r"\s+", " ", regex=True).str.strip()
giulia_df = giulia_df.drop_duplicates(subset=["FoodName_clean"])

# off file data cleaning
import json
import re
import pandas as pd

off_clean = []
with open("/content/drive/MyDrive/UPF-HFI/nova/openfoodfacts-popular-24.json", "r", encoding="utf-8") as f:
    for line in f:
        try:
            entry = json.loads(line)
            if not isinstance(entry, dict):
                continue

            name = entry.get("product_name") or entry.get("abbreviated_product_name")
            nova = entry.get("nova_group")

            if name and nova:
                name_clean = re.sub(r"[^\w\s]", " ", name.lower())
                name_clean = re.sub(r"\s+", " ", name_clean).strip()
                off_clean.append({"FoodName_clean": name_clean, "NOVA": nova})

        except json.JSONDecodeError:
            continue

off_df = pd.DataFrame(off_clean).drop_duplicates(subset=["FoodName_clean"])



In [4]:
ndns_df.to_csv("NDNS_clean.csv", index=False)
giulia_df.to_csv("Giulia_clean.csv", index=False)
off_df.to_csv("OFF_clean.csv", index=False)


# 1. Keyword Matching

## 1.1 Description Part

In [5]:
def match_nova_by_description(text):
    text = str(text).lower().strip()

    # === NOVA 1: water ===
    if any(w in text for w in ["tap water", "still water", "filtered water", "plain water"]):
        if "flavour" not in text:
            return 1, "plain water (description)"

    # === NOVA 1: plain===
    if any(w in text for w in ["semi skimmed milk", "skimmed milk", "whole milk"]) and "flavour" not in text:
        return 1, "plain milk"
    if any(w in text for w in ["natural yoghurt", "fromage frais"]) and "flavour" not in text:
        return 1, "plain yoghurt"

    # === NOVA 1: raw/unprocessed ===
    # raw
    import re

    if re.search(r'\braw\b', text):
        return 1, "raw (word-bound)"

    # uncooked oat
    if re.search(r"\buncooked\b", text) and re.search(r"\boat(s)?\b", text):
        return 1, "raw cereal: oats (description)"


    # === NOVA 3: homemade/light-processed ===
    if any(w in text for w in ["homemade", "home made"]):
        return 3, "homemade"
    if any(w in text for w in ["boiled", "mashed potato", "baked potato", "jacket potato"]):
        return 3, "boiled/baked/jacket"

    # === NOVA 4: sachet porridge ===
    if "porridge sachet" in text or ("porridge" in text and "oat so simple" in text):
        return 4, "sachet porridge (description)"

    # === NOVA 4: takeaway  ===
    if "takeaway" in text or "take away" in text:
        return 4, "takeaway food"

    # === NOVA 4: sweets/dessert/snack ===
    if any(w in text for w in ["jam", "conserve", "marmalade", "chocolate spread", "ice cream topping", "marzipan"]):
        return 4, "spread/syrup"
    if any(w in text for w in ["cracker", "savoury biscuit", "cheddar biscuit", "cream cracker"]):
        return 4, "processed snack"
    if any(w in text for w in ["sweets", "gums", "jelly", "boiled sweets", "mints", "liquorice", "popcorn"]):
        return 4, "sweet snack"
    if any(w in text for w in ["ice cream", "dessert", "milkshake"]):
        return 4, "processed dessert"
    if any(w in text for w in ["margarine", "clover spread", "flora"]):
        return 4, "processed fat"
    if "flavoured milk" in text or "chocolate milk" in text:
        return 4, "flavoured milk"
    if "ketchup" in text and "home made" not in text:
        return 4, "processed ketchup"
    if "instant" in text and "porridge" not in text:
        return 4, "instant food"
    # === NOVA 4: takeaway 快餐类 ===
    if "takeaway" in text or "take away" in text:
        return 4, "takeaway food"

    return None, None


## 1.2 Group Part

In [6]:
def match_nova_by_group(group, description):
    group = str(group).lower().strip()
    description = str(description).lower().strip()

    # === NOVA 1: group water ===
    if group.strip() in ["water", "tap water", "filtered water"]:
        return 1, "water (group)"

    # === NOVA 1: milk/yoghurt ===
    if "fresh fruit" in group:
        return 1, "fruit (group)"
    if "dried fruit" in group:
        return 1, "dried fruit (group)"
    if "vegetables" in group and "fried" not in group:
        return 1, "vegetables (group)"
    if any(word in group for word in ["semi skimmed milk", "skimmed milk", "whole milk"]):
        if "flavour" not in description and "fruit" not in description:
            return 1, "milk (group)"
    if any(word in group for word in ["natural yoghurt", "fromage frais"]):
        if "flavour" not in description and "fruit" not in description:
            return 1, "yoghurt/plain dairy (group)"

    # === NOVA 3: some fat ===
    if any(w in group for w in ["olive oil", "rapeseed oil", "sunflower oil", "vegetable oil", "butter"]):
        return 3, "culinary fat/oil (group)"

    # === NOVA 4 ===
    if any(w in group for w in ["margarine", "fat spread", "flora", "dairy fat spreads", "hard marg"]):
        return 4, "processed fat (group)"
    if any(w in group for w in ["jam", "conserve", "marmalade"]):
        return 4, "preserves (group)"
    if "other breakfast cereals" in group or "muesli" in group or "bran flakes" in group:
        return 4, "processed cereal (group)"

    return None, None


In [7]:
print(intake_df.columns)


Index(['SurveyID', 'UserID', 'Source', 'Starttime', 'Submissiontime',
       'Timetocomplete', 'Cookingoilused', 'Diet', 'Foodamount',
       'Reasonforunusualfoodamount',
       ...
       'Modification_Identification', 'discontinued', 'NDNS_Checks',
       'UserID_specific', 'Day', 'weekday', 'ratio', 'UserID_clean',
       'Foodgroupen_clean', 'Descriptionen_clean'],
      dtype='object', length=168)


## 1.3 Combined Rule Matching & Application

In [8]:
def match_nova(row):
    description = row["Descriptionen_clean"]
    group = row["Foodgroupen_clean"]

    # try description
    nova, reason = match_nova_by_description(description)
    if nova is not None:
        return pd.Series([nova, "description: " + reason])

    # fallback to group
    nova, reason = match_nova_by_group(group, description)
    if nova is not None:
        return pd.Series([nova, "group: " + reason])

    return pd.Series([None, None])


In [9]:
intake_df[["NOVA_step1", "match_reason"]] = intake_df.apply(match_nova, axis=1)

## 1.4 Save outcome_step1

In [10]:
output_path = "/content/drive/MyDrive/UPF-HFI/outcome/intake_with_nova_step1.csv"

cols_to_save = [
    "Descriptionen",
    "Descriptionen_clean",
    "Foodgroupen",
    "Foodgroupen_clean",
    "NOVA_step1",
    "match_reason"
]
intake_df[cols_to_save].to_csv(output_path, index=False)


# 2. TF-IDF Matching

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Step 2.1: Merge three NOVA control files and clean them
ndns_df['source'] = 'NDNS'
giulia_df['source'] = 'Giulia'
off_df['source'] = 'OFF'

nova_all = pd.concat([ndns_df, giulia_df, off_df], axis=0)
nova_all = nova_all.drop_duplicates(subset='FoodName_clean').reset_index(drop=True)

# Step 2.2: Select only the samples missing from NOVA_step1
to_match_mask = intake_df['NOVA_step1'].isna()
intake_tfidf_df = intake_df[to_match_mask].copy()

# Step 2.3: TF-IDF vectorization and similarity calculation
vectorizer = TfidfVectorizer()
tfidf_intake = vectorizer.fit_transform(intake_tfidf_df['Descriptionen_clean'])
tfidf_nova = vectorizer.transform(nova_all['FoodName_clean'])

cos_sim = cosine_similarity(tfidf_intake, tfidf_nova)
best_match_idx = cos_sim.argmax(axis=1)
best_scores = cos_sim.max(axis=1)

# Step 2.4: Only keep the matching results with score >= 0.85
score_threshold = 0.85
valid_mask = best_scores >= score_threshold

intake_tfidf_df.loc[valid_mask, 'TFIDF_score'] = best_scores[valid_mask]
intake_tfidf_df.loc[valid_mask, 'TFIDF_match_name'] = nova_all.iloc[best_match_idx[valid_mask]]['FoodName_clean'].values
intake_tfidf_df.loc[valid_mask, 'NOVA_step2'] = nova_all.iloc[best_match_idx[valid_mask]]['NOVA'].values
intake_tfidf_df.loc[valid_mask, 'Match_source'] = nova_all.iloc[best_match_idx[valid_mask]]['source'].values

# Step 2.5: Merge the results back to the original intake_df
intake_df.loc[intake_tfidf_df.index, 'TFIDF_score'] = intake_tfidf_df['TFIDF_score']
intake_df.loc[intake_tfidf_df.index, 'TFIDF_match_name'] = intake_tfidf_df['TFIDF_match_name']
intake_df.loc[intake_tfidf_df.index, 'NOVA_step2'] = intake_tfidf_df['NOVA_step2']
intake_df.loc[intake_tfidf_df.index, 'Match_source'] = intake_tfidf_df['Match_source']


In [12]:
output_path = "/content/drive/MyDrive/UPF-HFI/outcome/intake_with_step2_tfidf.csv"

columns_to_save = [
    'Descriptionen', 'Descriptionen_clean',
    'NOVA_step1', 'match_reason',
    'NOVA_step2', 'TFIDF_score', 'TFIDF_match_name', 'Match_source'
]
intake_df[columns_to_save].to_csv(output_path, index=False)


# 3. SBERT

In [13]:
from sentence_transformers import SentenceTransformer, util

# Step 3.1: 筛选 Step1 和 Step2 都未匹配的样本
mask_sbert = intake_df['NOVA_step1'].isna() & intake_df['NOVA_step2'].isna()
intake_sbert_df = intake_df[mask_sbert].copy()

# Step 3.2: 加载 SBERT 模型
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Step 3.3: 编码为句向量
intake_embeddings = model.encode(intake_sbert_df['Descriptionen_clean'].tolist(), convert_to_tensor=True)
nova_embeddings = model.encode(nova_all['FoodName_clean'].tolist(), convert_to_tensor=True)

# Step 3.4: 相似度矩阵
cos_scores = util.pytorch_cos_sim(intake_embeddings, nova_embeddings)
best_match_scores, best_match_indices = cos_scores.max(dim=1)

# Step 3.5: 阈值控制
threshold = 0.8
score_array = best_match_scores.cpu().numpy()
index_array = best_match_indices.cpu().numpy()

# 高分匹配行
high_mask = score_array >= threshold
low_mask = ~high_mask

# 填入高分匹配结果
intake_sbert_df.loc[high_mask, 'SBERT_score'] = score_array[high_mask]
intake_sbert_df.loc[high_mask, 'SBERT_match_name'] = nova_all.iloc[index_array[high_mask]]['FoodName_clean'].values
intake_sbert_df.loc[high_mask, 'NOVA_step3'] = nova_all.iloc[index_array[high_mask]]['NOVA'].values
intake_sbert_df.loc[high_mask, 'SBERT_match_source'] = nova_all.iloc[index_array[high_mask]]['source'].values

# 对低分匹配，仅记录匹配信息，但不填入 NOVA
intake_sbert_df.loc[low_mask, 'SBERT_score'] = score_array[low_mask]
intake_sbert_df.loc[low_mask, 'SBERT_match_name'] = nova_all.iloc[index_array[low_mask]]['FoodName_clean'].values
intake_sbert_df.loc[low_mask, 'SBERT_flag'] = 'low_confidence'

# 合并回主表
intake_df.loc[intake_sbert_df.index, ['SBERT_score', 'SBERT_match_name', 'NOVA_step3', 'SBERT_match_source', 'SBERT_flag']] = \
    intake_sbert_df[['SBERT_score', 'SBERT_match_name', 'NOVA_step3', 'SBERT_match_source', 'SBERT_flag']]


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [15]:
# 设置输出路径
output_path = "/content/drive/MyDrive/UPF-HFI/outcome/intake_with_step3_sbert.csv"

# 保存 SBERT 匹配结果（保留 step1~4 及关键分数和来源）
columns_to_save = [
    'Descriptionen', 'Descriptionen_clean',
    'NOVA_step1', 'match_reason',
    'NOVA_step2', 'TFIDF_score', 'TFIDF_match_name',
    'NOVA_step3', 'SBERT_score', 'SBERT_match_name',
    'Match_source', 'SBERT_match_source', 'SBERT_flag'
]
intake_df[columns_to_save].to_csv(output_path, index=False)


In [16]:
# 计算未匹配的样本数量
unmatched_mask = (
    intake_df['NOVA_step1'].isna() &
    intake_df['NOVA_step2'].isna() &
    intake_df['NOVA_step3'].isna()
)
unmatched_count = unmatched_mask.sum()

# 总样本数量
total_count = len(intake_df)
matched_count = total_count - unmatched_count
matched_percent = matched_count / total_count * 100

print(f"已匹配样本数量：{matched_count} / {total_count}")
print(f"匹配覆盖率：{matched_percent:.2f}%")
print(f"尚未匹配样本数量：{unmatched_count}")


已匹配样本数量：18175 / 22217
匹配覆盖率：81.81%
尚未匹配样本数量：4042


# 4. 验证冲突

In [18]:
# === Step 4: 检查模型冲突（仅比较 Step2 vs Step3 的 NOVA） ===
conflict_df = intake_df[
    intake_df['NOVA_step2'].notna() &
    intake_df['NOVA_step3'].notna() &
    (intake_df['NOVA_step2'] != intake_df['NOVA_step3'])
].copy()

# 添加冲突说明列
conflict_df['Conflict_type'] = 'Step2 vs Step3 NOVA mismatch'

# 保存冲突详情
conflict_df.to_csv(
    "/content/drive/MyDrive/UPF-HFI/outcome/step4_conflict_TFIDF_SBERT.csv",
    index=False
)

print(f"仅模型冲突分析完成，共发现 {len(conflict_df)} 条 TF-IDF vs SBERT 的 NOVA 分类冲突样本。")


仅模型冲突分析完成，共发现 0 条 TF-IDF vs SBERT 的 NOVA 分类冲突样本。


# 5. 合并以上所有

In [21]:
print(intake_df.columns.tolist())


['SurveyID', 'UserID', 'Source', 'Starttime', 'Submissiontime', 'Timetocomplete', 'Cookingoilused', 'Diet', 'Foodamount', 'Reasonforunusualfoodamount', 'Proxy', 'ProxyIssues', 'MealIndex', 'MealID', 'Mealname', 'Mealtime', 'Foodsource', 'FoodIndex', 'Searchterm', 'FoodID', 'Intake24foodcode', 'Descriptionen', 'Descriptionlocal', 'Nutrienttablename', 'Nutrienttablecode', 'Foodgroupcode', 'Foodgroupen', 'Foodgrouplocal', 'Readymeal', 'Brand', 'Asservedweightfactor', 'Servingsizegml', 'Servingimage', 'Leftoversgml', 'Leftoversimage', 'Portionsizegml', 'Reasonableamount', 'MissingfoodID', 'Missingfooddescription', 'Missingfoodportionsize', 'Missingfoodleftovers', 'Subgroupcode', 'Water', 'Totalnitrogen', 'Nitrogenconversionfactor', 'Protein', 'Fat', 'Carbohydrate', 'Energykcal', 'EnergykJ', 'Alcohol', 'Englystfibre', 'Starch', 'Totalsugars', 'AOAC', 'Nonmilkextrinsicsugars', 'Intrinsicandmilksugars', 'Glucose', 'Fructose', 'Maltose', 'Lactose', 'Sucrose', 'OthersugarsUK', 'FSTablesugar', '

In [23]:
# Step 5.0 提前重命名（避免旧列名出错）
rename_map = {
    "NOVA_step1": "NOVA_keyword",
    "match_reason": "Keyword_reason",
    "NOVA_step2": "NOVA_tfidf",
    "TFIDF_match_name": "TFIDF_match",
    "Match_source": "TFIDF_source",
    "NOVA_step3": "NOVA_sbert",
    "SBERT_match_name": "SBERT_match",
    "SBERT_match_source": "SBERT_source"
}
intake_df.rename(columns=rename_map, inplace=True)

# Step 5.1 检查模型间矛盾匹配
intake_df['Model_disagree_flag'] = (
    intake_df['TFIDF_match'].notna() &
    intake_df['SBERT_match'].notna() &
    (intake_df['TFIDF_match'] != intake_df['SBERT_match'])
)

intake_df['NOVA_disagree_flag'] = (
    intake_df['Model_disagree_flag'] &
    (intake_df['NOVA_tfidf'] != intake_df['NOVA_sbert'])
)

intake_df['Decision_flag'] = intake_df['NOVA_disagree_flag'].map(
    lambda x: "manual_review" if x else "auto_accept"
)

# Step 5.2 选择最终可信值：Keyword > SBERT > TF-IDF
final_mask = intake_df['Decision_flag'] == "auto_accept"
intake_df['NOVA_final'] = None

intake_df.loc[final_mask & intake_df['NOVA_keyword'].notna(), 'NOVA_final'] = intake_df['NOVA_keyword']
intake_df.loc[final_mask & intake_df['NOVA_final'].isna() & intake_df['NOVA_sbert'].notna(), 'NOVA_final'] = intake_df['NOVA_sbert']
intake_df.loc[final_mask & intake_df['NOVA_final'].isna() & intake_df['NOVA_tfidf'].notna(), 'NOVA_final'] = intake_df['NOVA_tfidf']

# Step 5.3 来源与理由（含 Rule-based → keyword）
def get_final_source(row):
    if pd.notna(row['NOVA_keyword']):
        return 'keyword'
    elif pd.notna(row['NOVA_sbert']):
        return row['SBERT_source']
    elif pd.notna(row['NOVA_tfidf']):
        return row['TFIDF_source']
    else:
        return None

def get_final_reason(row):
    if pd.notna(row['NOVA_keyword']):
        return row['Keyword_reason']
    elif pd.notna(row['NOVA_sbert']):
        return 'SBERT matched'
    elif pd.notna(row['NOVA_tfidf']):
        return 'TF-IDF matched'
    else:
        return 'Unmatched'

intake_df['Final_source'] = intake_df.apply(get_final_source, axis=1)
intake_df['Final_reason'] = intake_df.apply(get_final_reason, axis=1)

# Step 5.4 保存最终结果
final_cols = [
    "Descriptionen", "Descriptionen_clean",
    "NOVA_keyword", "Keyword_reason",
    "NOVA_tfidf", "TFIDF_score", "TFIDF_match", "TFIDF_source",
    "NOVA_sbert", "SBERT_score", "SBERT_match", "SBERT_source",
    "Model_disagree_flag", "NOVA_disagree_flag", "Decision_flag",
    "NOVA_final", "Final_source", "Final_reason"
]

intake_df[final_cols].to_csv("/content/drive/MyDrive/UPF-HFI/outcome/intake_with_nova_final.csv", index=False)


In [24]:
slim_cols = [
    "Descriptionen",
    "NOVA_final",
    "Final_reason"
]

intake_df[slim_cols].to_csv(
    "/content/drive/MyDrive/UPF-HFI/outcome/intake_with_nova_final_slim.csv",
    index=False
)


# 6. Unmatched-18.19%(4042)

In [25]:
# 添加 Failure_reason 列到总表中（如果尚未加过）
def get_failure_reason(row):
    if pd.notna(row['NOVA_final']):
        return None
    elif row['Decision_flag'] == 'manual_review':
        return 'Model disagreement'
    elif pd.isna(row['NOVA_keyword']) and pd.isna(row['NOVA_tfidf']) and pd.isna(row['NOVA_sbert']):
        return 'No match found'
    else:
        return 'Conflict unresolved'

intake_df['Failure_reason'] = intake_df.apply(get_failure_reason, axis=1)

# 筛选出 unmatched 行
unmatched_df = intake_df[intake_df['NOVA_final'].isna()].copy()

# 保存 unmatched 文件
unmatched_df[["Descriptionen", "Failure_reason"]].to_csv(
    "/content/drive/MyDrive/UPF-HFI/outcome/intake_unmatched_summary.csv",
    index=False
)


In [26]:
from sentence_transformers import SentenceTransformer, util
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Step 6.1: 获取未匹配样本（NOVA_final为空）
unmatched_df = intake_df[intake_df['NOVA_final'].isna()].copy()

# Step 6.2: TF-IDF fallback 匹配（score ≥ 0.6）
vectorizer = TfidfVectorizer()
tfidf_intake = vectorizer.fit_transform(unmatched_df['Descriptionen_clean'])
tfidf_nova = vectorizer.transform(nova_all['FoodName_clean'])
cos_sim = cosine_similarity(tfidf_intake, tfidf_nova)

tfidf_best_idx = cos_sim.argmax(axis=1)
tfidf_best_scores = cos_sim.max(axis=1)
tfidf_thresh = 0.6
tfidf_mask = tfidf_best_scores >= tfidf_thresh

unmatched_df.loc[tfidf_mask, 'NOVA_final_fallback'] = nova_all.iloc[tfidf_best_idx[tfidf_mask]]['NOVA'].values
unmatched_df.loc[tfidf_mask, 'Fallback_source'] = nova_all.iloc[tfidf_best_idx[tfidf_mask]]['source'].values
unmatched_df.loc[tfidf_mask, 'Fallback_reason'] = 'TF-IDF fallback'

# Step 6.3: SBERT fallback 匹配（score ≥ 0.6，仅限 TF-IDF 未命中的）
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
sbert_input = unmatched_df['Descriptionen_clean'].tolist()
sbert_embed_intake = model.encode(sbert_input, convert_to_tensor=True)
sbert_embed_nova = model.encode(nova_all['FoodName_clean'].tolist(), convert_to_tensor=True)

sbert_sim = util.pytorch_cos_sim(sbert_embed_intake, sbert_embed_nova)
sbert_best_scores, sbert_best_idx = sbert_sim.max(dim=1)

sbert_thresh = 0.6
# bool_mask 是 np.array，转成 Series 保留索引对齐
bool_mask = sbert_best_scores.cpu().numpy() >= sbert_thresh
sbert_mask = pd.Series(bool_mask, index=unmatched_df.index)

# 用索引对齐方式获取匹配结果
matched_indices = sbert_best_idx.cpu().numpy()[sbert_mask.values]
matched_rows = unmatched_df[sbert_mask]

# 填入 fallback 匹配结果（通过 .iloc 避免 index 错位）
unmatched_df.loc[matched_rows.index, 'NOVA_final_fallback'] = nova_all.iloc[matched_indices]['NOVA'].values
unmatched_df.loc[matched_rows.index, 'Fallback_source'] = nova_all.iloc[matched_indices]['source'].values
unmatched_df.loc[matched_rows.index, 'Fallback_reason'] = 'SBERT fallback'

In [27]:
# Step 6.4: 筛选出匹配成功的 fallback 行并保存
fallback_df = unmatched_df[unmatched_df['NOVA_final_fallback'].notna()][[
    "Descriptionen", "NOVA_final_fallback", "Fallback_source", "Fallback_reason"
]]

fallback_df.to_csv(
    "/content/drive/MyDrive/UPF-HFI/outcome/intake_unmatched_fallback_matched.csv",
    index=False
)

print(f"Fallback 匹配完成，共补全 {len(fallback_df)} 条 unmatched 样本")


Fallback 匹配完成，共补全 3809 条 unmatched 样本


In [28]:
# Step6.5: 获取 fallback 失败的 unmatched 样本
failed_fallback_df = unmatched_df[unmatched_df['NOVA_final_fallback'].isna()]

# 统计 Descriptionen 出现次数，并重命名列
failure_counts = (
    failed_fallback_df['Descriptionen']
    .value_counts()
    .reset_index()
)
failure_counts.columns = ['Descriptionen', 'Count']

# 按出现次数降序显示
print("所有 fallback 失败样本的 Descriptionen（含重复）:")
print(failure_counts.head(30))  # 你可以调整数量


所有 fallback 失败样本的 Descriptionen（含重复）:
                                        Descriptionen  Count
0                                        Vimto, still     61
1                                  Pom Bears (crisps)     52
2                                            Pringles     18
3                                         Cheesy mash     15
4                                               Fanta     10
5                             Magnum classic or white     10
6                                            Smarties      8
7                        Lollipops (e.g. Chupa Chups)      7
8                                   Fox's party rings      6
9                            Walker Sensations crisps      6
10                                          Poppadums      5
11                                         Fanta Zero      4
12                                     Malteser bunny      3
13                                      7 up / Sprite      3
14                                             